In [109]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [110]:
weatherpy_csv = "../WeatherPy/Output/WeatherPy.csv"
df = pd.read_csv(weatherpy_csv)
df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,"Pangai, TO",-19.8000,-174.3500,77.81,72,100,8.43
1,"Bengkulu, ID",-3.8004,102.2655,77.02,89,99,6.76
2,"Hambantota, LK",6.1241,81.1185,81.72,79,79,2.82
3,"Kaoma, ZM",-14.7833,24.8000,67.86,68,80,5.37
4,"Carnarvon, AU",-24.8667,113.6333,68.07,83,40,3.44


In [82]:
# Specify values from DataFrame
locations = df[["Latitude","Longitude"]]
humidity = df["Humidity"]
cities = df["City"]
temp = df["Temperature"]
cloud = df["Cloudiness"]
wind = df["Wind Speed"]
df.count()

City           618
Latitude       618
Longitude      618
Temperature    618
Humidity       618
Cloudiness     618
Wind Speed     618
dtype: int64

In [83]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations, weights=humidity, 
    dissipating=False, max_intensity=100,point_radius=1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [84]:
# Ideal vacation conditions
# temperature range (min, max)
temp_range = [75,82]
# wind speed range (min, max)
wind_range = [0,10]
# preferred cloudiness range (min, max)
cloud_range = [0,10]

In [111]:
# Locate values
ideal_df = df.loc[
    (temp >= temp_range[0]) & (temp <= temp_range[1]) & 
    (wind >= wind_range[0]) & (wind <= wind_range[1]) &
    (cloud >= cloud_range[0]) & (cloud <= cloud_range[1])
]
ideal_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
29,"Umm Lajj, SA",25.0213,37.2685,78.73,48,0,6.22
103,"Cabo San Lucas, MX",22.8909,-109.9124,76.87,70,0,6.91
112,"Sao Filipe, CV",14.8961,-24.4956,77.52,63,2,8.63
266,"Inhambane, MZ",-23.8650,35.3833,75.31,88,10,8.52
368,"Vallenar, CL",-28.5708,-70.7581,78.31,28,0,6.06
398,"Grand Gaube, MU",-20.0064,57.6608,76.37,69,3,5.01
500,"Beira, MZ",-19.8436,34.8389,76.89,84,3,5.61
520,"Guaymas, MX",27.9333,-110.9000,79.95,34,0,0.00
571,"Constitucion, MX",23.9912,-104.6749,75.13,22,0,3.00


In [196]:
vacation_df = ideal_df[['City', 'Latitude', 'Longitude']].copy()
vacation_df['Hotel Name'] = ""
vacation_df['Hotel Lat'] = ""
vacation_df['Hotel Lon'] = ""

vacation_df.head()

,City,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Lon
29,"Umm Lajj, SA",25.0213,37.2685,,,
103,"Cabo San Lucas, MX",22.8909,-109.9124,,,
112,"Sao Filipe, CV",14.8961,-24.4956,,,
266,"Inhambane, MZ",-23.8650,35.3833,,,
368,"Vallenar, CL",-28.5708,-70.7581,,,


In [173]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=[YOURAPIKEY]&location=-33.8670522,151.1957362&radius=5000&type=restaurant

In [199]:
q_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in vacation_df.iterrows():
    city = row["City"]
    lat = row["Latitude"]
    lon = row["Longitude"]
    url = f'{q_url}?key={g_key}&location={lat},{lon}&radius=5000&type=lodging'
    response = requests.get(url).json()
    
    try:
        print(f"Retrieving Results for {city}.")
        hotel_name = response["results"][0]['name']
        print(f"Found hotel: {hotel_name}")
        vacation_df.loc[index, 'Hotel Name'] = hotel_name
        vacation_df.loc[index, 'Hotel Lat'] = response["results"][0]['geometry']['location']['lat']
        vacation_df.loc[index, 'Hotel Lon'] = response["results"][0]['geometry']['location']['lng']
    except:
        pass


Retrieving Results for Umm Lajj, SA.
Found hotel: Moon light Furnished Units
Retrieving Results for Cabo San Lucas, MX.
Found hotel: Hotel Tesoro Los Cabos
Retrieving Results for Sao Filipe, CV.
Found hotel: Hotel Xaguate
Retrieving Results for Inhambane, MZ.
Found hotel: Oceano
Retrieving Results for Vallenar, CL.
Found hotel: Humacao Bed & Breakfast
Retrieving Results for Grand Gaube, MU.
Found hotel: Veranda Paul et Virginie Hotel & Spa
Retrieving Results for Beira, MZ.
Found hotel: Beira Terrace Hotel
Retrieving Results for Guaymas, MX.
Found hotel: Holiday Inn Express Guaymas, an IHG Hotel
Retrieving Results for Constitucion, MX.
Found hotel: Hotel Posada San Agustin


In [200]:
vacation_df.head()

,City,Latitude,Longitude,Hotel Name,Hotel Lat,Hotel Lon
29,"Umm Lajj, SA",25.0213,37.2685,Moon light Furnished Units,25.0174,37.2718
103,"Cabo San Lucas, MX",22.8909,-109.9124,Hotel Tesoro Los Cabos,22.8815,-109.912
112,"Sao Filipe, CV",14.8961,-24.4956,Hotel Xaguate,14.8996,-24.4982
266,"Inhambane, MZ",-23.8650,35.3833,Oceano,-23.8614,35.3467
368,"Vallenar, CL",-28.5708,-70.7581,Humacao Bed & Breakfast,-28.5732,-70.7597
